In [1]:
# Import packages
from transformers import AutoModel, AutoTokenizer
import torch

In [2]:
# Load BioBERT Model
tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")
model = AutoModel.from_pretrained("dmis-lab/biobert-v1.1")

/u/sswee/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Encode words
def encode_text(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    
    # Forward pass to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Extract the [CLS] token representation (sentence embedding)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    
    return cls_embedding

# Example usage
text = "The patient's heart is fine."
vector = encode_text(text)
print(vector.shape)  # Should be (1, 768) for BioBERT-base

torch.Size([1, 768])
